<a href="https://colab.research.google.com/github/miiyu-fujita/MAGIC/blob/main/shapes_GUI_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import / Install necessary libraries

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
! pip install gradio
import gradio as gr

Upload the model
- download the model (it will download as a zip) from the drive link in the repo. 
- upload the model zip file to this environment (/content folder), then run this cell to unzip the files

In [ ]:
! unzip "/content/shapes_model-20230303T232527Z-001.zip" -d "/model"

Load the model using tensorflow
- ensure that the path keras is loading the model from matches the path to which you have unzipped the model files

In [ ]:
model = keras.models.load_model('/model/shapes_model')

Build the GUI

In [ ]:
# accept zip files , extract images, present in gallery & predict (2 outputs: gallery of images & predicted zip file)
import zipfile 
import glob
import os
import io
from os import path 
import shutil 

# ----- YOUR IMAGES MUST BE .PNG, OR YOU HAVE TO CHANGE THE GLOB ------
# zip file input -> extract pics to show gallery of uploaded images
def zip_to_gallery(file): # does it work with the file itself? - YESSSS
  print(file.name)
  with zipfile.ZipFile(file.name, 'r') as zip_obj:
    zip_obj.extractall('/input_images')
  return glob.glob('/input_images/**/*.png')

def all_predictions(file): 
  # initialize the parent directory and 3 child directories 
  if not path.exists('/predictions'):
    os.mkdir('/predictions')
    os.mkdir('/predictions/circle')
    os.mkdir('/predictions/square')
    os.mkdir('/predictions/star')
    os.mkdir('/predictions/triangle')
  # get image paths of extracted image files
  image_paths = zip_to_gallery(file) # HARD CODED PATH - USED TO WORK WITH ANY FILE PATH ??W??DSAC><SDKMCSKJDN
  for ipath in image_paths:
    image = tf.keras.utils.load_img(ipath, target_size=(200, 200))
    image_array = tf.keras.utils.img_to_array(image)
    image_array = tf.expand_dims(image_array, 0)
    predictions = model.predict(image_array)
    score = tf.nn.softmax(predictions[0])
    class_names = ['circle', 'square', 'star', 'triangle']

    prediction_label = class_names[np.argmax(score)]
    # here, based on label, save the image to the folder "Igneous", "Metamorphic" or "Sedimentary"
    if prediction_label == 'circle':
      shutil.copy(ipath, '/predictions/circle')
    elif prediction_label == 'square':
      shutil.copy(ipath, '/predictions/square')
    elif prediction_label == 'star':
      shutil.copy(ipath, '/predictions/star')
    else: 
      shutil.copy(ipath, '/predictions/triangle')
  shutil.make_archive('predictions', 'zip', '/', 'predictions')
  return image_paths, "/content/predictions.zip"

def get_circle():
  # assumes main page has an uploaded zip file with predictions 
  # returns a Gallery & zip file 
  rock_1_image_paths = glob.glob('/predictions/circle/*.png')
  shutil.make_archive('circle', 'zip', '/', '/predictions/circle')
  return rock_1_image_paths, "/content/circle.zip"

def get_square():
  rock_2_image_paths = glob.glob('/predictions/square/*.png')
  shutil.make_archive('square', 'zip', '/', '/predictions/square')
  return rock_2_image_paths, "/content/square.zip"

def get_star():
  rock_3_image_paths = glob.glob('/predictions/star/*.png')
  shutil.make_archive('star', 'zip', '/', '/predictions/star')
  return rock_3_image_paths, "/content/star.zip"

def get_triangle():
  rock_4_image_paths = glob.glob('/predictions/triangle/*.png')
  shutil.make_archive('triangle', 'zip', '/', '/predictions/triangle')
  return rock_4_image_paths, "/content/triangle.zip"



all_rock_demo = gr.Interface(all_predictions,
                    gr.File(file_types=[".zip"]),
                    outputs=[gr.Gallery(), "file"],
                    title="MAGIC",
                    description="upload a zip file!")
circle_demo = gr.Interface(get_circle,
                         None,
                         outputs=[gr.Gallery(), "file"],
                         title="MAGIC",
                         description="these are Circles :)")
square_demo = gr.Interface(get_square,
                         None,
                         outputs=[gr.Gallery(), "file"],
                         title="MAGIC",
                         description="these are Squares :)")
star_demo = gr.Interface(get_star,
                         None,
                         outputs=[gr.Gallery(), "file"],
                         title="MAGIC",
                         description="these are Stars :)")
triangle_demo = gr.Interface(get_triangle,
                         None,
                         outputs=[gr.Gallery(), "file"],
                         title="MAGIC",
                         description="these are Triangles :)")



demo = gr.TabbedInterface([all_rock_demo, circle_demo, square_demo, star_demo, triangle_demo], ["All rocks", "Circle", "Square", "Star", "Triangle"])


Launch the GUI

In [ ]:
demo.launch(debug=True)